In [ ]:
#importing requiered libraries and data set
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
data = pd.read_csv("/content/drive/MyDrive/pet_finder/petfinder-mini.csv")
data.head()

In [ ]:
# cheking if there is null parameters
data.isnull().sum()

In [ ]:
# storeing  `AdoptionSpeed` column in variable Y ; storeing `Description` column in variable D to be able to use it in `CountVectorizer()`; storeing the reamaining columns in variable X

X = data.drop(['AdoptionSpeed', 'Description'], axis=1)
Y = data['AdoptionSpeed']
D = data['Description']



In [ ]:
# filling the null parameters
D = D.fillna('')

In [ ]:
# cheking if there is any null left
D.isna().sum()

0

In [ ]:
# defining `CountVectorizer` and calling it on our D variable
cv = CountVectorizer()
D_transformed = cv.fit_transform(D)
D_transformed

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
# defining categorical features list for `OneHotEncoder0` and transforming our X variable
categorical_features = ["Type","Breed1","Gender","Color1",
                        "Color2","MaturitySize","FurLength","Vaccinated",
                        "Sterilized","Health"]
one_hot=OneHotEncoder()
transformer=ColumnTransformer([("one_hot",
                               one_hot,
                               categorical_features)],
                              remainder="passthrough")
OH_X = transformer.fit_transform(X)

In [ ]:
from scipy.sparse import hstack
# apprnding variable D to OH_X vaiable 
OH_X_with_text = hstack([OH_X, D_transformed])

In [ ]:
# seting up our training and test data
xtrain, xtest, ytrain, ytest = train_test_split(OH_X_with_text, Y, test_size=0.2)
# defining a base model
model_1 = DecisionTreeClassifier()
model_1.get_params()

In [ ]:
#fitting the base model and testing it
model_1.fit(xtrain, ytrain)
predictions = model_1.score(xtest, ytest);

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
# ploting the resualt of our base model with confusion matrix
y_preds = model_1.predict(xtest)
accuracy = model_1.score(xtest, ytest)

print(f"model_1 Accuracy: { accuracy * 100}%")


cm = confusion_matrix(y_true=ytest, y_pred=y_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2","3","4"])
disp.plot(cmap='Blues', values_format='d')

plt.title('Confusion Matrix model_1')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestClassifier
# defining a second model
model_2 = RandomForestClassifier(n_jobs=-1,verbose=1)
model_2.get_params()

In [ ]:
# fitting new model and testing it
model_2.fit(xtrain, ytrain)
predictions = model_2.score(xtest, ytest);

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
#ploting the resualt of the second with confusion matrix
y_preds = model_2.predict(xtest)
accuracy = model_2.score(xtest, ytest)

print(f"Model_2 Accuracy: { accuracy * 100}%")


cm = confusion_matrix(y_true=ytest, y_pred=y_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2","3","4"])
disp.plot(cmap='Blues', values_format='d')

plt.title('Confusion Matrix mode_2')
plt.show()

In [ ]:
# defining a library for `GridSearchCV`
grid = {'n_estimators': [10,100,1000],
       'max_depth': [None,5,10,20],
       'max_features': ["sqrt","log2"],
        'min_samples_split' : [2,4],
        'min_samples_leaf' : [1,2]}


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# defining `evaluate_preds` function to get accuracy_score, precision_score, recall_score and f1_score of the model

def evaluate_preds(y_true, y_preds):
    accuracy=accuracy_score(y_true,y_preds,average='macro')
    percesion=precision_score(y_true,y_preds,average='macro')
    recall=recall_score(y_true,y_preds,average='macro')
    f1=f1_score(y_true,y_preds)
    metrics_dict = {'accuracy' : round(accuracy,2),
                   "percesion": round( percesion,2),
                   "recall":round(recall,2),
                    'f1' : round(f1,2)
                   }
    print(f'ACC :{ accuracy*100:2f}%')
    print(f'percesion: {percesion:2f}')
    print(f'recall: {recall:2f}')
    print(f'f1: {f1:2f}')
    return metrics_dict

In [ ]:
from sklearn.model_selection import GridSearchCV
# defining `GridSearchCV` model and fitting it
gs_clf = GridSearchCV(estimator=model_2,param_grid=grid,cv=2,verbose=1)
gs_clf.fit(xtrain,ytrain);


In [ ]:
# getting the parameters of the model 
gs_clf.best_params_

In [ ]:
import pickle
# saving our model
with open("/content/drive/MyDrive/pet_finder/model_2_random_forest_Grid_search_best_param.pkl", 'wb') as file:
    pickle.dump(model_2, file)


In [ ]:
# loading the model
model_2_GS_params_loaded = pickle.load(open('/content/drive/MyDrive/pet_finder/model_2_random_forest_Grid_search_best_param.pkl','rb'))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# chenging the average for `evaluate_preds` function and defining a new function as `evaluate_preds_weighted` 
def evaluate_preds_weighted(y_true, y_preds):
    accuracy = accuracy_score(y_true, y_preds)
    precision = precision_score(y_true, y_preds, average='weighted')  # Change average to 'micro', 'macro', or 'weighted'
    recall = recall_score(y_true, y_preds, average='weighted')  # Change average to 'micro', 'macro', or 'weighted'
    f1 = f1_score(y_true, y_preds, average='weighted')  # Change average to 'micro', 'macro', or 'weighted'

    metrics_dict = {
        'accuracy': round(accuracy, 2),
        'precision': round(precision, 2),
        'recall': round(recall, 2),
        'f1': round(f1, 2)
    }

    print(f'ACC: {accuracy * 100:.2f}%')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1: {f1:.2f}')

    return metrics_dict


In [ ]:
#makeing prediction with loaded model and geting accuracy_score, precision_score, recall_score and f1_score with new function
model_2_GS_params_loaded_y_preds = model_2_GS_params_loaded.predict(xtest)

model_2_GS_params_loaded_metrics= evaluate_preds_weighted(ytest, model_2_GS_params_loaded_y_preds)

In [ ]:
# ploting the model resaults using confusion matrix
print(f'Model accuracy on test set : {model_2_GS_params_loaded.score(xtest,ytest)*100}%')

# Display the confusion matrix
cm = confusion_matrix(y_true=ytest, y_pred=model_2_GS_params_loaded_y_preds)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["0","1","2","3","4"])
disp.plot(cmap='Blues', values_format='d')

plt.title('Confusion Matrix mode_2_GridSearchCV')
plt.show();

In [ ]:
import pickle
# saving model
with open("/content/drive/MyDrive/pet_finder/model_2_random_forest_Grid_search_best_param_88%.pkl", 'wb') as file:
    pickle.dump(model_2_GS_params_loaded, file)